In [1]:
# !!!!!!!!! DO NOT RUN IN MINERL CONDA ENVIRONMENT - this environment is very fragile and minerl is easily borken with chagned dependenceies but this code will want to update its dependencies and break it!
if False:
    !pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117
    !pip install opencv-python 
    !pip install git+https://github.com/openai/CLIP.git
    !pip install scikit-learn

In [3]:
import clip
from PIL import Image
import torch as th
import cv2
import random
from sklearn import svm as sklearn_svm
import pickle
import random

In [5]:
th.seed=0
random.seed(0)

In [6]:
# LOAD CLIP
device = "cuda" if th.cuda.is_available() else "cpu"
model, preprocess = clip.load("RN50x64", device=device)
model.to(device)

CLIP(
  (visual): ModifiedResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)


In [8]:
# LOAD DATASET

with open('spamham_samples/frames_labels.txt') as file:
    frames_labels = file.readlines()
    random.shuffle(frames_labels)


# load labels. ONLY LOAD 1 FRAME PER VIDEO
links = []
labels = []
for i in range(len(frames_labels)):
    link = frames_labels[i].split(',')[0]
    if not link in links:
        links.append(link)
        label = frames_labels[i].split(',')[1]
        labels.append(label)

num_samples = len(frames_labels)

# LOAD & ENCODE IMAGES
# from images, create image features from CLIP
x_image_features = []
y_labels = []
missing=0
for i in range(num_samples):
    try:
        image_name = 'spamham_samples/frames/' + ''.join(frames_labels[i].split(',')[0:-1]) + '.jpeg'
        with Image.open(image_name) as image:
            image = preprocess(image).unsqueeze(0).to(device)
        with th.no_grad():
            x_image_features.append(model.encode_image(image))
        y_labels.append(labels[i])

        #display progress
        if i%(num_samples//100)==0:
            print('=',end='')
    except:
        missing+=1
        pass
x_image_features = th.cat(x_image_features)
print('missing',missing)
num_samples -= missing
print(num_samples)


============================================================================================missing 516
7570


In [10]:
print(len(y_labels))
print(x_image_features.shape)

7570
torch.Size([7570, 1024])


In [43]:
# save image feature dataset for faster use later just in case
#th.save(x_image_features, 'x_image_features_3')
#th.save(y_labels, 'y_labels_3')

x_image_features = th.load('x_image_features_3')
y_labels = th.load('y_labels_3')

print(x_image_features.shape)
print(len(y_labels))

torch.Size([7570, 1024])
7570


In [44]:
# turn to CPU train/test features for SVM
import numpy as np
split=0.9
#y_labels = np.asarray(y_labels)
#y_labels[y_labels==3]=2
#y_labels = y_labels.tolist()
x_train = x_image_features[:int(num_samples*split)].cpu()
y_train = y_labels[:int(num_samples*split)]
x_test = x_image_features[int(num_samples*split):].cpu()
y_test = y_labels[int(num_samples*split):]

In [45]:
### INITIALIZE SVM
svm = sklearn_svm.SVC(kernel='rbf', gamma='scale', C=8)
#svm = pickle.load(open('SVM4', 'rb'))

In [46]:
svm.fit(x_train, y_train)

SVC(C=8)

In [47]:
# FIT SVM TO IMAGE FEATURES -> LABELS
svm.score(x_test, y_test)

0.750330250990753

In [48]:
# for final training, train on whole dataset
x_train = x_image_features.cpu()
y_train = y_labels
svm.fit(x_train, y_train)
pickle.dump(svm, open('SVM4', 'wb'))